In [16]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

In [17]:
rsna = pd.read_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\RSNA.csv')
ddsm = pd.read_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM.csv')
cmmd = pd.read_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CMMD.csv')
cdd = pd.read_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\cdd_cesm.csv')
cdd = cdd[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
cmmd = cmmd[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
rsna = rsna[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
ddsm = ddsm[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]

In [18]:
class Evaluate:
    def __init__(self, meta_pro, name):
        self.al_data = meta_pro.reset_index()
        self.name = name
    def quatity_cc_mlo(self):
        print("So luong CC và MLO")
        print(self.al_data.view.value_counts())
    def lelf_right(self):
        print("So luong L và R")
        print(self.al_data.groupby(['cancer', 'view', 'laterality']).size())
    def show_one(self, par):
        print(self.al_data[self.al_data.patient_id == par])
    def quanlity_cancer(self):
        sl = len(self.al_data[self.al_data.cancer == 1])
        print(f"So luong img cancer: {sl}")
        # print(self.al_data[self.al_data.cancer == 1])
    def show(self):
        ls = len(self.al_data)
        par = len(self.al_data.patient_id.unique())
        print(f'Dataset {self.name} \n Số lượng image: {ls} \n Số luong patient: {par}')
        self.quatity_cc_mlo()
        self.lelf_right()
        self.quanlity_cancer()

In [19]:
ev = Evaluate(rsna, 'rsna')
ev.show()

Dataset rsna 
 Số lượng image: 49968 
 Số luong patient: 11913
So luong CC và MLO
view
CC     24984
MLO    24984
Name: count, dtype: int64
So luong L và R
cancer  view  laterality
0       CC    L             12216
              R             12250
        MLO   L             12216
              R             12250
1       CC    L               262
              R               256
        MLO   L               262
              R               256
dtype: int64
So luong img cancer: 1036


In [20]:
ev = Evaluate(cmmd, 'cmmd')
ev.show()

Dataset cmmd 
 Số lượng image: 3738 
 Số luong patient: 1772
So luong CC và MLO
view
CC     1869
MLO    1869
Name: count, dtype: int64
So luong L và R
cancer  view  laterality
0       CC    L             280
              R             273
        MLO   L             280
              R             273
1       CC    L             707
              R             609
        MLO   L             707
              R             609
dtype: int64
So luong img cancer: 2632


In [21]:
ev = Evaluate(cdd, 'cdd')
ev.show()

Dataset cdd 
 Số lượng image: 671 
 Số luong patient: 267
So luong CC và MLO
view
MLO    336
CC     335
Name: count, dtype: int64
So luong L và R
cancer  view  laterality
0       CC    L             82
              R             90
        MLO   L             75
              R             93
1       CC    L             93
              R             70
        MLO   L             98
              R             70
dtype: int64
So luong img cancer: 331


In [22]:
ev = Evaluate(ddsm, 'ddsm')
ev.show()

Dataset ddsm 
 Số lượng image: 5124 
 Số luong patient: 1137
So luong CC và MLO
view
CC     2562
MLO    2562
Name: count, dtype: int64
So luong L và R
cancer  view  laterality
0       CC    L             602
              R             602
        MLO   L             602
              R             602
1       CC    L             679
              R             679
        MLO   L             679
              R             679
dtype: int64
So luong img cancer: 2716


In [8]:
group_rsna = rsna.groupby(['cancer', 'view', 'laterality', 'image_id']).size()
group_ddsm = ddsm.groupby(['cancer', 'view', 'laterality', 'image_id']).size()
group_cmmd = cmmd.groupby(['cancer', 'view', 'laterality', 'image_id']).size()
group_cdd = cdd.groupby(['cancer', 'view', 'laterality', 'image_id']).size()

In [9]:
group_rsna

cancer  view  laterality  image_id  
0       CC    L           392070        1
                          600749        1
                          896533        1
                          1315983       1
                          1362161       1
                                       ..
1       MLO   R           2124340393    1
                          2126262062    1
                          2126290731    1
                          2137661476    1
                          2142591939    1
Length: 49968, dtype: int64

### CMMD

In [9]:
len(cmmd.image_id.unique())

3738

In [10]:
cmmd

,patient_id,image_id,view,laterality,cancer
0,D1-0001,D1-0001-1-2,CC,R,0
1,D1-0001,D1-0001-1-1,MLO,R,0
2,D1-0002,D1-0002-1-2,CC,L,0
3,D1-0002,D1-0002-1-1,MLO,L,0
4,D1-0003,D1-0003-1-2,CC,L,0
...,...,...,...,...,...
3733,D2-0747,D2-0747-1-3,MLO,R,1
3734,D2-0748,D2-0748-1-4,CC,R,1
3735,D2-0748,D2-0748-1-3,MLO,R,1
3736,D2-0749,D2-0749-1-2,CC,L,1


In [11]:
cmmd.rename(columns={'Unnamed: 0', 'index'})

TypeError: 'set' object is not callable

In [11]:
group_cmmd[0]['CC']["L"].index

Index(['D1-0002-1-2', 'D1-0003-1-2', 'D1-0004-1-2', 'D1-0006-1-2',
       'D1-0008-1-2', 'D1-0009-1-2', 'D1-0011-1-2', 'D1-0012-1-2',
       'D1-0013-1-2', 'D1-0014-1-2',
       ...
       'D1-1041-1-2', 'D1-1059-1-2', 'D1-1220-1-2', 'D1-1673-1-2',
       'D2-0090-1-2', 'D2-0092-1-2', 'D2-0158-1-2', 'D2-0259-1-2',
       'D2-0265-1-2', 'D2-0701-1-2'],
      dtype='object', name='image_id', length=280)

In [12]:
import random


In [13]:
250/4

62.5

In [14]:
0.5*4

2.0

In [15]:
cmmd_cc_l0_id_image = random.sample(list(group_cmmd[0]['CC']["L"].index), 62)
cmmd_cc_l0 = cmmd[cmmd.image_id.isin(cmmd_cc_l0_id_image)].reset_index()
cmmd_cc_l0

,index,patient_id,image_id,view,laterality,cancer
0,4,D1-0003,D1-0003-1-2,CC,L,0
1,16,D1-0009,D1-0009-1-2,CC,L,0
2,24,D1-0013,D1-0013-1-2,CC,L,0
3,50,D1-0025,D1-0025-1-2,CC,L,0
4,56,D1-0028,D1-0028-1-2,CC,L,0
...,...,...,...,...,...,...
57,1496,D1-1010,D1-1010-1-2,CC,L,0
58,1504,D1-1014,D1-1014-1-2,CC,L,0
59,1536,D1-1029,D1-1029-1-2,CC,L,0
60,2536,D2-0158,D2-0158-1-2,CC,L,0


In [11]:
class process_label:
    def __init__(self, data):
        self.group_data = data.groupby(['cancer', 'view', 'laterality', 'image_id']).size()
        self.data = data
    def xx(self, view, lat, quantity, cancer = 0):
        xxx_cc_r0_id_image = random.sample(list(self.group_data[cancer][view][lat].index), quantity)
        xxx_cc_r0 = self.data[self.data.image_id.isin(xxx_cc_r0_id_image)].reset_index()
        return xxx_cc_r0[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
    def process(self, quantity, name):
        
        ######## CC ############
        CC0L = self.xx('CC', 'L', quantity, 0)
        CC0R = self.xx('CC', 'R', quantity, 0)
        
        CC1L = self.xx('CC', 'L', quantity, 1)
        CC1R = self.xx('CC', 'R', quantity, 1)
        
        ######## MLO ############
        
        MLO0L = self.xx('MLO', 'L', quantity, 0)
        MLO0R = self.xx('MLO', 'R', quantity, 0)
        
        MLO1L = self.xx('MLO', 'L', quantity, 1)
        MLO1R = self.xx('MLO', 'R', quantity, 1)
        
        meta = pd.concat([CC0L, CC0R, CC1L, CC1R, MLO0L, MLO0R,MLO1L,  MLO1R])
        
        ev= Evaluate(meta, name)
        ev.show()
        return meta

In [17]:
p_cmmd = process_label(cmmd)
label_cmmd = p_cmmd.process(62, 'CMMD')
label_cmmd.head(5)

Dataset CMMD 
 Số lượng image: 496 
 Số luong patient: 434
So luong CC và MLO
view
CC     248
MLO    248
Name: count, dtype: int64
So luong L và R
cancer  view  laterality
0       CC    L             62
              R             62
        MLO   L             62
              R             62
1       CC    L             62
              R             62
        MLO   L             62
              R             62
dtype: int64
So luong img cancer: 248


,patient_id,image_id,view,laterality,cancer
0,D1-0002,D1-0002-1-2,CC,L,0
1,D1-0004,D1-0004-1-2,CC,L,0
2,D1-0008,D1-0008-1-2,CC,L,0
3,D1-0011,D1-0011-1-2,CC,L,0
4,D1-0019,D1-0019-1-2,CC,L,0


In [18]:
p_ddsm = process_label(ddsm)
label_ddsm = p_ddsm.process(62, 'DDSM')
label_ddsm.head(5)

Dataset DDSM 
 Số lượng image: 496 
 Số luong patient: 425
So luong CC và MLO
view
CC     248
MLO    248
Name: count, dtype: int64
So luong L và R
cancer  view  laterality
0       CC    L             62
              R             62
        MLO   L             62
              R             62
1       CC    L             62
              R             62
        MLO   L             62
              R             62
dtype: int64
So luong img cancer: 248


,patient_id,image_id,view,laterality,cancer
0,0014DDSM,2752,CC,L,0
1,0024DDSM,2776,CC,L,0
2,0031DDSM,2792,CC,L,0
3,0044DDSM,2836,CC,L,0
4,0052DDSM,2864,CC,L,0


In [19]:
p_cdd = process_label(cdd)
label_cdd = p_cdd.process(63, 'CDD')
label_cdd.head(5)

Dataset CDD 
 Số lượng image: 504 
 Số luong patient: 243
So luong CC và MLO
view
CC     252
MLO    252
Name: count, dtype: int64
So luong L và R
cancer  view  laterality
0       CC    L             63
              R             63
        MLO   L             63
              R             63
1       CC    L             63
              R             63
        MLO   L             63
              R             63
dtype: int64
So luong img cancer: 252


,patient_id,image_id,view,laterality,cancer
0,5CDD,P5-L-DM-CC,CC,L,0
1,16CDD,P16-L-DM-CC,CC,L,0
2,21CDD,P21-L-DM-CC,CC,L,0
3,30CDD,P30-L-DM-CC,CC,L,0
4,32CDD,P32-L-DM-CC,CC,L,0


In [23]:
p_rsna = process_label(rsna)
label_rsna = p_rsna.process(63, 'rsna')
label_rsna.head(5)

NameError: name 'process_label' is not defined

### Move image to STANDARD DATA

In [24]:
cdd = cdd.reset_index()

In [13]:
cnt = 0

In [15]:
from cdd import *

In [112]:
cddmove = move(cdd)
cddmove.my_method(list(cdd.index))

In [20]:
label_ddsm

NameError: name 'label_ddsm' is not defined

In [25]:
ddsm = pd.read_csv(r"E:\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Cau_truc_data\meta_ddsm.csv").drop(['Unnamed: 0', 'index'], axis=1)
ddsm

,patient_id,image_id,view,laterality,cancer,fullPath
0,0001DDSM,0,CC,L,1,Cancer\0001\C_0001_1.LEFT_CC.jpg
1,0001DDSM,1,MLO,L,1,Cancer\0001\C_0001_1.LEFT_MLO.jpg
2,0001DDSM,2,CC,R,1,Cancer\0001\C_0001_1.RIGHT_CC.jpg
3,0001DDSM,3,MLO,R,1,Cancer\0001\C_0001_1.RIGHT_MLO.jpg
4,0002DDSM,4,CC,L,1,Cancer\0002\C_0002_1.LEFT_CC.jpg
...,...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0,Normal\4607\D_4607_1.RIGHT_MLO.jpg
5120,4608DDSM,5120,CC,L,0,Normal\4608\D_4608_1.LEFT_CC.jpg
5121,4608DDSM,5121,MLO,L,0,Normal\4608\D_4608_1.LEFT_MLO.jpg
5122,4608DDSM,5122,CC,R,0,Normal\4608\D_4608_1.RIGHT_CC.jpg


In [26]:
lab_ddsm = ddsm
lab_ddsm

,patient_id,image_id,view,laterality,cancer,fullPath
0,0001DDSM,0,CC,L,1,Cancer\0001\C_0001_1.LEFT_CC.jpg
1,0001DDSM,1,MLO,L,1,Cancer\0001\C_0001_1.LEFT_MLO.jpg
2,0001DDSM,2,CC,R,1,Cancer\0001\C_0001_1.RIGHT_CC.jpg
3,0001DDSM,3,MLO,R,1,Cancer\0001\C_0001_1.RIGHT_MLO.jpg
4,0002DDSM,4,CC,L,1,Cancer\0002\C_0002_1.LEFT_CC.jpg
...,...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0,Normal\4607\D_4607_1.RIGHT_MLO.jpg
5120,4608DDSM,5120,CC,L,0,Normal\4608\D_4608_1.LEFT_CC.jpg
5121,4608DDSM,5121,MLO,L,0,Normal\4608\D_4608_1.LEFT_MLO.jpg
5122,4608DDSM,5122,CC,R,0,Normal\4608\D_4608_1.RIGHT_CC.jpg


In [27]:
lab_ddsm.fullPath =lab_ddsm.fullPath.apply(lambda x: x.replace('jpg', 'png'))
lab_ddsm

,patient_id,image_id,view,laterality,cancer,fullPath
0,0001DDSM,0,CC,L,1,Cancer\0001\C_0001_1.LEFT_CC.png
1,0001DDSM,1,MLO,L,1,Cancer\0001\C_0001_1.LEFT_MLO.png
2,0001DDSM,2,CC,R,1,Cancer\0001\C_0001_1.RIGHT_CC.png
3,0001DDSM,3,MLO,R,1,Cancer\0001\C_0001_1.RIGHT_MLO.png
4,0002DDSM,4,CC,L,1,Cancer\0002\C_0002_1.LEFT_CC.png
...,...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0,Normal\4607\D_4607_1.RIGHT_MLO.png
5120,4608DDSM,5120,CC,L,0,Normal\4608\D_4608_1.LEFT_CC.png
5121,4608DDSM,5121,MLO,L,0,Normal\4608\D_4608_1.LEFT_MLO.png
5122,4608DDSM,5122,CC,R,0,Normal\4608\D_4608_1.RIGHT_CC.png


In [ ]:
lab_ddsm.

In [28]:
# E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16/
from cdd import *

In [29]:
import shutil

In [25]:
moveddsm = move(lab_ddsm)

moveddsm.my_method(list(lab_ddsm.index))

In [ ]:
E:\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Cau_truc_data\move_cc_mlo.py

In [35]:
for i in range(len(ddsm)):
    par = ddsm.patient_id[i]
    view = ddsm.view[i]
    lat = ddsm.laterality[i]
    idimg = ddsm.image_id[i]
    # name = ddsm.image_id[i].replace('-', '_')
    pat = ddsm.fullPath[i]
    path = f"E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\{pat}"
    name = os.path.basename(path)
    # print(path)
    shutil.move(path, 'E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM')
    os.rename(f'E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM\{name}', f'E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM\{view}_{idimg}.png')

### Move Label 

### XONG C

In [28]:
from cdd import *

In [29]:
# label_cmmd = label_cmmd.reset_index()
label_cmmd

,patient_id,image_id,view,laterality,cancer
0,D1-0002,D1-0002-1-2,CC,L,0
1,D1-0004,D1-0004-1-2,CC,L,0
2,D1-0008,D1-0008-1-2,CC,L,0
3,D1-0011,D1-0011-1-2,CC,L,0
4,D1-0019,D1-0019-1-2,CC,L,0
...,...,...,...,...,...
57,D2-0710,D2-0710-1-3,MLO,R,1
58,D2-0723,D2-0723-1-3,MLO,R,1
59,D2-0729,D2-0729-1-3,MLO,R,1
60,D2-0732,D2-0732-1-3,MLO,R,1


In [30]:
label_cmmd = label_cmmd.reset_index()

In [31]:
meta_cmmd = pd.read_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\TheChineseMammographyDatabase\TheChineseMammographyDatabase\meta_main.csv")
meta_cmmd

,Unnamed: 0,index,patient_id,image_id,laterality,cancer,path,view
0,0,0,D1-0001,D1-0001_1-1,R,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
1,1,1,D1-0001,D1-0001_1-2,R,0,D:/OneDrive - Industrial University of HoChiMi...,CC
2,2,2,D1-0002,D1-0002_1-1,L,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
3,3,3,D1-0002,D1-0002_1-2,L,0,D:/OneDrive - Industrial University of HoChiMi...,CC
4,4,4,D1-0003,D1-0003_1-1,L,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
...,...,...,...,...,...,...,...,...
3736,3736,3736,D2-0747,D2-0747_1-4,R,1,D:/OneDrive - Industrial University of HoChiMi...,CC
3737,3737,3737,D2-0748,D2-0748_1-3,R,1,D:/OneDrive - Industrial University of HoChiMi...,MLO
3738,3738,3738,D2-0748,D2-0748_1-4,R,1,D:/OneDrive - Industrial University of HoChiMi...,CC
3739,3739,3739,D2-0749,D2-0749_1-1,L,1,D:/OneDrive - Industrial University of HoChiMi...,MLO


In [32]:
meta_cmmd.image_id = meta_cmmd.image_id.apply(lambda x: x.replace('_', '-'))
meta_cmmd.image_id[1]

'D1-0001-1-2'

In [33]:
meta_cmmd.path = meta_cmmd.path.apply(lambda x: x.replace('D:/OneDrive - Industrial University of HoChiMinh City/', 'E:/'))
meta_cmmd.path[1]

'E:/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-NA-70244\\1-2.dcm'

In [34]:
label_cmmd

,index,patient_id,image_id,view,laterality,cancer
0,0,D1-0002,D1-0002-1-2,CC,L,0
1,1,D1-0004,D1-0004-1-2,CC,L,0
2,2,D1-0008,D1-0008-1-2,CC,L,0
3,3,D1-0011,D1-0011-1-2,CC,L,0
4,4,D1-0019,D1-0019-1-2,CC,L,0
...,...,...,...,...,...,...
491,57,D2-0710,D2-0710-1-3,MLO,R,1
492,58,D2-0723,D2-0723-1-3,MLO,R,1
493,59,D2-0729,D2-0729-1-3,MLO,R,1
494,60,D2-0732,D2-0732-1-3,MLO,R,1


In [35]:
def procees_dcm(path, path_m):
    
        dicom = pydicom.dcmread(path)
        # try : 
        img = dicom.pixel_array
        # except:
        #     # print(self.get_info())
           
        #     f.write(self.get_info())
        #     return 
            
        img = apply_windowing(img, dicom)
        img = (img - img.min()) / (img.max() - img.min())

        if dicom.PhotometricInterpretation == "MONOCHROME1":  
            img = 1 - img
        image = np.uint8(img * 255)
        imageio.imwrite(path_m, img)
        return image

In [38]:
len(label_rsna)

504

In [41]:
label_cdd = label_cdd.reset_index()
label_cdd

,index,patient_id,image_id,view,laterality,cancer
0,0,5CDD,P5-L-DM-CC,CC,L,0
1,1,16CDD,P16-L-DM-CC,CC,L,0
2,2,21CDD,P21-L-DM-CC,CC,L,0
3,3,30CDD,P30-L-DM-CC,CC,L,0
4,4,32CDD,P32-L-DM-CC,CC,L,0
...,...,...,...,...,...,...
499,58,306CDD,P306-R-DM-MLO,MLO,R,1
500,59,311CDD,P311-R-DM-MLO,MLO,R,1
501,60,313CDD,P313-R-DM-MLO,MLO,R,1
502,61,318CDD,P318-R-DM-MLO,MLO,R,1


In [ ]:
import tqdm

In [42]:
for i in label_cdd.index:
    par = label_cdd.patient_id[i]
    view = label_cdd.view[i]
    lat = label_cdd.laterality[i]
    idimg = label_cdd.image_id[i]
    path = f"E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CDD-CESM\{view}_{idimg}.png"
    # print(patj)
    if os.path.exists(path) == False:
        # print(meta_cmmd[(meta_cmmd.view == view) & (meta_cmmd.image_id == idimg)])
        print(path)
        # img = procees_dcm(meta_cmmd[(meta_cmmd.view == view) & (meta_cmmd.image_id == idimg)].path.values[0], path)
        # x=1
        # try:
        #     imageio.imwrite( f'E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\{view}_{idimg}_{par}.png', img)
        # except:
        #     print(path)
    else:
        par = label_cdd.patient_id[i]
        view = label_cdd.view[i]
        lat = label_cdd.laterality[i]
        idimg = label_cdd.image_id[i]
        path = f"E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CDD-CESM\{view}_{idimg}.png"
        img = cv2.imread(path)
        try:
            imageio.imwrite( f'E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\{view}_{idimg}_{par}.png', img)
        except:
            print(path)

In [ ]:
"E:\WORKBASE\Project-rsna-breast-cancer-detection\TheChineseMammographyDatabase (2)\TheChineseMammographyDatabase\TheChineseMammographyDatabase\CMMD\D1-0014\07-18-2010-NA-NA-09653\1.000000-NA-74216\1-1.dcm"

In [54]:
len(label_rsna) + len(label_ddsm) + len(label_cmmd)

1496

In [55]:
label_ddsm = label_ddsm.reset_index()

In [64]:
cnt = 0
for i in label_ddsm.index:
    par = label_ddsm.patient_id[i]
    view = label_ddsm.view[i]
    lat = label_ddsm.laterality[i]
    idimg = label_ddsm.image_id[i]
    path = f"E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\{view}_{idimg}_{par}.png"
    # print(patj)
    if os.path.exists(path) == True:
        # print(meta_cmmd[(meta_cmmd.view == view) & (meta_cmmd.image_id == idimg)])
        print(path)
        cnt+=1
    #    CC_D1-0003-1-2_D1-0003.png?

E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_3032_0119DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_3112_0148DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_3616_0328DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_4740_4508DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_3310_0241DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_4458_1966DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_4806_4524DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_4886_4546DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_4934_4558DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_4954_4563DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_5002_4576DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_720_0194DDSM.png
E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\CC_996_1469DDSM.png
E:\WORKBASE\Pr

In [65]:
cnt

46

In [66]:
316+ 504+ 46

866

In [51]:
1496 - len(glob.glob('E:\WORKBASE\Project-rsna-breast-cancer-detection\Label\*.png')) 

62

In [ ]:
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0014/07-18-2010-NA-NA-09653/1.000000-NA-74216\1-1.dcm

In [130]:
procees_dcm("E:/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase (2)/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0014/07-18-2010-NA-NA-09653/1.000000-NA-74216/1-1.dcm")

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [48]:
meta_label = pd.concat([label_ddsm, label_rsna, label_cdd, label_cmmd]).reset_index()
meta_label

,level_0,patient_id,image_id,view,laterality,cancer,index
0,0,0014DDSM,2752,CC,L,0,NaN
1,1,0024DDSM,2776,CC,L,0,NaN
2,2,0031DDSM,2792,CC,L,0,NaN
3,3,0044DDSM,2836,CC,L,0,NaN
4,4,0052DDSM,2864,CC,L,0,NaN
...,...,...,...,...,...,...,...
1995,491,D2-0710,D2-0710-1-3,MLO,R,1,57.0
1996,492,D2-0723,D2-0723-1-3,MLO,R,1,58.0
1997,493,D2-0729,D2-0729-1-3,MLO,R,1,59.0
1998,494,D2-0732,D2-0732-1-3,MLO,R,1,60.0


In [49]:
meta_label = meta_label[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
meta_label

,patient_id,image_id,view,laterality,cancer
0,0014DDSM,2752,CC,L,0
1,0024DDSM,2776,CC,L,0
2,0031DDSM,2792,CC,L,0
3,0044DDSM,2836,CC,L,0
4,0052DDSM,2864,CC,L,0
...,...,...,...,...,...
1995,D2-0710,D2-0710-1-3,MLO,R,1
1996,D2-0723,D2-0723-1-3,MLO,R,1
1997,D2-0729,D2-0729-1-3,MLO,R,1
1998,D2-0732,D2-0732-1-3,MLO,R,1


In [ ]:
meta_label.to_csv('meta_label.csv')

In [4]:
meta_label = pd.read_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Cau_truc_data\meta_label.csv')
meta_label

,Unnamed: 0,patient_id,image_id,view,laterality,cancer
0,0,0014DDSM,2752,CC,L,0
1,1,0024DDSM,2776,CC,L,0
2,2,0031DDSM,2792,CC,L,0
3,3,0044DDSM,2836,CC,L,0
4,4,0052DDSM,2864,CC,L,0
...,...,...,...,...,...,...
1995,1995,D2-0710,D2-0710-1-3,MLO,R,1
1996,1996,D2-0723,D2-0723-1-3,MLO,R,1
1997,1997,D2-0729,D2-0729-1-3,MLO,R,1
1998,1998,D2-0732,D2-0732-1-3,MLO,R,1


In [19]:
len(meta_label.image_id.unique())

2000

In [16]:
group_meta_label = meta_label.groupby(['cancer', 'view', 'laterality', 'image_id']).size()
group_meta_label

cancer  view  laterality  image_id    
0       CC    L           1001606983      1
                          1053554375      1
                          1065799163      1
                          1133284324      1
                          1152713088      1
                                         ..
1       MLO   R           P66-R-DM-MLO    1
                          P85-R-DM-MLO    1
                          P89-R-DM-MLO    1
                          P90-R-DM-MLO    1
                          P96-R-DM-MLO    1
Length: 2000, dtype: int64

In [29]:
ccr0 = meta_label[meta_label.image_id.isin(random.sample(list(group_meta_label[0]['CC']['R'].index), 25))]
ccr1 = meta_label[meta_label.image_id.isin(random.sample(list(group_meta_label[1]['CC']['R'].index), 25))]

ccL0 = meta_label[meta_label.image_id.isin(random.sample(list(group_meta_label[0]['CC']['L'].index), 25))]
ccL1 = meta_label[meta_label.image_id.isin(random.sample(list(group_meta_label[1]['CC']['L'].index), 25))]

MLOr0 = meta_label[meta_label.image_id.isin(random.sample(list(group_meta_label[0]['MLO']['R'].index), 25))]
MLOr1 = meta_label[meta_label.image_id.isin(random.sample(list(group_meta_label[1]['MLO']['R'].index), 25))]

MLOL0 = meta_label[meta_label.image_id.isin(random.sample(list(group_meta_label[0]['MLO']['L'].index), 25))]
MLOL1 = meta_label[meta_label.image_id.isin(random.sample(list(group_meta_label[1]['MLO']['L'].index), 25))]


In [33]:
val = pd.concat([ccr0, ccr1, ccL0, ccL1, MLOr0, MLOr1, MLOL0, MLOL1] ).reset_index()
val = val[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
val

,patient_id,image_id,view,laterality,cancer
0,0030DDSM,2790,CC,R,0
1,0566DDSM,4182,CC,R,0
2,0610DDSM,4334,CC,R,0
3,1962DDSM,4442,CC,R,0
4,10925,506759176,CC,R,0
...,...,...,...,...,...
195,D1-1326,D1-1326-1-1,MLO,L,1
196,D1-1539,D1-1539-1-1,MLO,L,1
197,D1-1550,D1-1550-1-1,MLO,L,1
198,D2-0060,D2-0060-1-1,MLO,L,1


In [35]:
train = meta_label[~meta_label.image_id.isin(list(val.image_id))].reset_index()
train = train[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]

train

,patient_id,image_id,view,laterality,cancer
0,0014DDSM,2752,CC,L,0
1,0024DDSM,2776,CC,L,0
2,0031DDSM,2792,CC,L,0
3,0044DDSM,2836,CC,L,0
4,0052DDSM,2864,CC,L,0
...,...,...,...,...,...
1795,D2-0710,D2-0710-1-3,MLO,R,1
1796,D2-0723,D2-0723-1-3,MLO,R,1
1797,D2-0729,D2-0729-1-3,MLO,R,1
1798,D2-0732,D2-0732-1-3,MLO,R,1


In [37]:
val.to_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\split_data_lable\val.csv')
train.to_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\split_data_lable\train.csv')